In [0]:
import pandas as pd   #to use read_csv(), crosstab()
import numpy as np    #to use exponentiation : exp()
import statsmodels.api as sm  #to use the Logit()
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#import the dataset
traindata=pd.read_csv('credit_card_default_train.csv',sep=",")
testdata=pd.read_csv('credit_card_default_test.csv',sep=",")

In [0]:
traindata=traindata.replace(['M','F'], [1,-1]) 
testdata=testdata.replace(['M','F'], [1,-1]) 

In [0]:
traindata=traindata.replace(['Graduate','High School','Other'], [2,1,0]) 
testdata=testdata.replace(['Graduate','High School','Other'], [2,1,0]) 

In [0]:
traindata=traindata.replace(['Single'], [1]) 
testdata=testdata.replace(['Single'], [1]) 

In [0]:
traindata['AGE']=traindata['AGE'].replace(['Less than 30','31-45','46-65','More than 65'], [1,2,3,4]) 
testdata['AGE']=testdata['AGE'].replace(['Less than 30','31-45','46-65','More than 65'], [1,2,3,4]) 


In [0]:
traindata['Balance_Limit_V1'] = traindata['Balance_Limit_V1'].map(lambda x : float(x[0:-1])*1000 if (x[-1]=='K') else float(x[0:-1])*1000000)
testdata['Balance_Limit_V1'] = testdata['Balance_Limit_V1'].map(lambda x : float(x[0:-1])*1000 if (x[-1]=='K') else float(x[0:-1])*1000000)

In [0]:
testdata[testdata.columns[12:24]]=testdata[testdata.columns[12:24]]/traindata.max()[traindata.columns[12:24]]
traindata[traindata.columns[12:24]]=traindata[traindata.columns[12:24]]/traindata.max()[traindata.columns[12:24]]


In [0]:
testdata['Balance_Limit_V1']=testdata['Balance_Limit_V1']/traindata.max()['Balance_Limit_V1']
traindata['Balance_Limit_V1']=traindata['Balance_Limit_V1']/traindata.max()['Balance_Limit_V1']


In [0]:
traindata['BIAS']=1
testdata['BIAS']=1

In [519]:
independentVar= ['Balance_Limit_V1',
                 'Gender',
                 'EDUCATION_STATUS',
                 'MARITAL_STATUS',
                 'AGE',
                 'PAY_JULY','PAY_AUG','PAY_SEP','PAY_OCT','PAY_NOV','PAY_DEC',
                 'DUE_AMT_JULY','DUE_AMT_AUG','DUE_AMT_SEP','DUE_AMT_OCT','DUE_AMT_NOV','DUE_AMT_DEC',
                 'PAID_AMT_JULY','PAID_AMT_AUG','PAID_AMT_SEP','PAID_AMT_OCT','PAID_AMT_NOV','PAID_AMT_DEC','BIAS']

X_train, y_train,X_test =traindata[independentVar],traindata['NEXT_MONTH_DEFAULT'],testdata[independentVar]
# X_train, y_train =traindata[independentVar],traindata['NEXT_MONTH_DEFAULT']

from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X_train,y_train,test_size=0.30)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
 

# define oversampling strategy
# over = RandomOverSampler(sampling_strategy=0.1)
# # fit and apply the transform
# X_train_res, y_train_res = over.fit_resample(X, y)
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.4)
# fit and apply the transform
X_train_res, y_train_res = under.fit_resample(X_train, y_train)

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train_res, y_train_res.ravel())





/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [520]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_res)

X_train_res = scaler.transform(X_train_res)
X_test = scaler.transform(X_test)


from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_res,y_train_res)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [521]:
y_pred = knn.predict(X_test)
y=[]
for i in y_pred:
  y.append([i])
ydata=pd.DataFrame(y,columns=['NEXT_MONTH_DEFAULT'])
ydata['Client_ID']=testdata['Client_ID']
print (ydata.groupby(['NEXT_MONTH_DEFAULT']).count())

output = ydata.to_csv('test.csv')


# from sklearn import metrics
# from sklearn.metrics import confusion_matrix
# confusion_matrix = confusion_matrix(y_test, y_pred)
# print(confusion_matrix)

                    Client_ID
NEXT_MONTH_DEFAULT           
0                        4089
1                        1911


In [0]:
# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))
